### Log
* xgboost

In [1]:
import argparse
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold
import shutil
import time
import gc
import random
import math
import torch
from torch.utils.data import DataLoader, Dataset
import logging
import transformers
from transformers import TrainingArguments, Trainer, DataCollatorForWholeWordMask
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel
from transformers.modeling_outputs  import BaseModelOutput,SequenceClassifierOutput
from torch import nn
from torch.optim import Adam, SGD, AdamW
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup, DataCollatorWithPadding
from datasets import load_metric, disable_progress_bar
import datasets
# imports the torch_xla package
import wandb
from torch.nn.parameter import Parameter
#os.environ["WANDB_DISABLED"] = "true"
from tqdm import tqdm
from spellchecker import SpellChecker
import re
import nltk
from nltk.corpus import stopwords
from collections import Counter
import spacy
import xgboost
import warnings
from dataclasses import dataclass, field
from typing import Optional

warnings.simplefilter('ignore')
logging.disable(logging.ERROR);
os.environ['TOKENIZER_PARALLELISM'] = 'false'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
disable_progress_bar()
tqdm.pandas()

In [2]:
import spellchecker
spellchecker.__version__

'0.7.2'

In [3]:
# set random seed
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
    
seed_everything(seed=42)

## Config

In [4]:
class CFG:
    pretraining = False
    load_pretrained = False
    input_path = './input/'
    input_type = '2'
    model_path = 'microsoft/deberta-v3-large' #  nghuyong/ernie-2.0-large-en studio-ousia/luke-large
    model_type = 'custom'
    scheduler = 'cosine'  # ['linear', 'cosine']
    batch_scheduler = True
    num_cycles = 0.5  # 1.5
    num_warmup_steps = 0
    max_input_length = 1600
    max_position_embeddings = 1600
    n_folds = 4
    folds = [1]
    epochs = 4  # 5
    # layer - wise larning rate 
    discriminative_learning_rate = False
    discriminative_learning_rate_num_groups = 1
    discriminative_learning_rate_decay_rate = 0.99
    # reinint layer
    reinit_layers = 0
    
#     encoder_lr = 5e-6
#     head_lr = 5e-6
    encoder_lr = 20e-6
    head_lr = 10e-5
    
    min_lr = 1e-7
    eps = 1e-7
    betas = (0.9, 0.999)
    weight_decay = 0
    dropout = 0
    num_fold = 5
    batch_size = 8
    seed = 42
    OUTPUT_DIR = './pretrain/'
    num_workers = 2
    device='cuda'
    print_freq = 100
    run_deberta_model = True;

    
@dataclass
class Config:
    model_name_or_path: Optional[str] = field(
        default="microsoft/deberta-v3-base",
        metadata={"help": "Model name or path"},
    )

    data_dir: Optional[str] = field(
        default="/kaggle/input/commonlit-evaluate-student-summaries",
        metadata={"help": "Data directory"},
    )

    max_seq_length: Optional[int] = field(
        default=CFG.max_input_length,
        metadata={"help": "Max sequence length"},
    )

    add_prompt_question: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt question into input"},
    )

    add_prompt_text: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt text into input"},
    )

    fold: Optional[int] = field(
        default=0,
        metadata={"help": "Fold"},
    )

    num_proc: Optional[int] = field(
        default=4,
        metadata={"help": "Number of processes"},
    )

    dropout: Optional[float] = field(
        default=0.,
        metadata={"help": "Amount of dropout to apply"},
    )
    max_position_embeddings: Optional[int] = field(
        default=CFG.max_input_length,
        metadata={"help": "Amount of dropout to apply"},
    )


def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

In [5]:
nltk.data.path.append('/root/workspace/commonlit/nltk')

In [6]:
class Preprocessor:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained(CFG.model_path);
        self.STOP_WORDS = set(stopwords.words('english'));
        
        self.spacy_ner_model = spacy.load('en_core_web_sm', )
        self.speller = SpellChecker()
        
    def count_text_length(self, df: pd.DataFrame, col:str) -> pd.Series:
        """ text length """
        tokenizer=self.tokenizer
        return df[col].progress_apply(lambda x: len(tokenizer.encode(x)))

    def word_overlap_count(self, row):
        """ intersection(prompt_text, text) """        
        def check_is_stop_word(word):
            return word in self.STOP_WORDS
        
        prompt_words = row['prompt_tokens']
        summary_words = row['summary_tokens']
        if self.STOP_WORDS:
            prompt_words = list(filter(check_is_stop_word, prompt_words))
            summary_words = list(filter(check_is_stop_word, summary_words))
        return len(set(prompt_words).intersection(set(summary_words)))
            
    def ngrams(self, token, n):
        # Use the zip function to help us generate n-grams
        # Concatentate the tokens into ngrams and return
        ngrams = zip(*[token[i:] for i in range(n)])
        return [" ".join(ngram) for ngram in ngrams]

    def ngram_co_occurrence(self, row, n: int):
        # Tokenize the original text and summary into words
        original_tokens = row['prompt_tokens']
        summary_tokens = row['summary_tokens']

        # Generate n-grams for the original text and summary
        original_ngrams = set(self.ngrams(original_tokens, n))
        summary_ngrams = set(self.ngrams(summary_tokens, n))

        # Calculate the number of common n-grams
        common_ngrams = original_ngrams.intersection(summary_ngrams)

        # # Optionally, you can get the frequency of common n-grams for a more nuanced analysis
        # original_ngram_freq = Counter(ngrams(original_words, n))
        # summary_ngram_freq = Counter(ngrams(summary_words, n))
        # common_ngram_freq = {ngram: min(original_ngram_freq[ngram], summary_ngram_freq[ngram]) for ngram in common_ngrams}

        return len(common_ngrams)
    
    def ner_overlap_count(self, row, mode:str):
        model = self.spacy_ner_model
        def clean_ners(ner_list):
            return set([(ner[0].lower(), ner[1]) for ner in ner_list])
        prompt = model(row['prompt_text'])
        summary = model(row['text'])

        if "spacy" in str(model):
            prompt_ner = set([(token.text, token.label_) for token in prompt.ents])
            summary_ner = set([(token.text, token.label_) for token in summary.ents])
        elif "stanza" in str(model):
            prompt_ner = set([(token.text, token.type) for token in prompt.ents])
            summary_ner = set([(token.text, token.type) for token in summary.ents])
        else:
            raise Exception("Model not supported")

        prompt_ner = clean_ners(prompt_ner)
        summary_ner = clean_ners(summary_ner)

        intersecting_ners = prompt_ner.intersection(summary_ner)
        
        ner_dict = dict(Counter([ner[1] for ner in intersecting_ners]))
        
        if mode == "train":
            return ner_dict
        elif mode == "test":
            return {key: ner_dict.get(key) for key in self.ner_keys}

    
    def quotes_count(self, row):
        summary = row['text']
        text = row['prompt_text']
        quotes_from_summary = re.findall(r'"([^"]*)"', summary)
        if len(quotes_from_summary)>0:
            return [quote in text for quote in quotes_from_summary].count(True)
        else:
            return 0

    def spelling(self, text):
        
        wordlist=text.split()
        amount_miss = len(list(self.speller.unknown(wordlist)))

        return amount_miss
    
    def run(self, 
            prompts: pd.DataFrame,
            summaries:pd.DataFrame,
            mode:str
        ) -> pd.DataFrame:
        
        # before merge preprocess
        prompts["prompt_length"] = prompts["prompt_text"].apply(
            lambda x: len(self.tokenizer.encode(x))
        )
        prompts["prompt_tokens"] = prompts["prompt_text"].apply(
            lambda x: self.tokenizer.convert_ids_to_tokens(
                self.tokenizer.encode(x), 
                skip_special_tokens=True
            )
        )

        summaries["summary_length"] = summaries["text"].apply(
            lambda x: len(self.tokenizer.encode(x))
        )
        summaries["summary_tokens"] = summaries["text"].apply(
            lambda x: self.tokenizer.convert_ids_to_tokens(
                self.tokenizer.encode(x), 
                skip_special_tokens=True
            )

        )
        summaries["splling_err_num"] = summaries["text"].progress_apply(self.spelling)

        # merge prompts and summaries
        input_df = summaries.merge(prompts, how="left", on="prompt_id")

        # after merge preprocess
        input_df['length_ratio'] = input_df['summary_length'] / input_df['prompt_length']
        
        input_df['word_overlap_count'] = input_df.progress_apply(self.word_overlap_count, axis=1)
        input_df['bigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence,args=(2,), axis=1 
        )
        input_df['trigram_overlap_count'] = input_df.progress_apply(
            self.ngram_co_occurrence, args=(3,), axis=1
        )
        
        # Crate dataframe with count of each category NERs overlap for all the summaries
        # Because it spends too much time for this feature, I don't use this time.
#         ners_count_df  = input_df.progress_apply(
#             lambda row: pd.Series(self.ner_overlap_count(row, mode=mode), dtype='float64'), axis=1
#         ).fillna(0)
#         self.ner_keys = ners_count_df.columns
#         ners_count_df['sum'] = ners_count_df.sum(axis=1)
#         ners_count_df.columns = ['NER_' + col for col in ners_count_df.columns]
#         # join ner count dataframe with train dataframe
#         input_df = pd.concat([input_df, ners_count_df], axis=1)
        
        input_df['quotes_count'] = input_df.progress_apply(self.quotes_count, axis=1)
        
        return input_df.drop(columns=["summary_tokens", "prompt_tokens"])
    
preprocessor = Preprocessor()   

### Read data and Create Folds

In [7]:
pdf = pd.read_csv(f"{CFG.input_path}/prompts_train.csv")
sdf = pd.read_csv(f"{CFG.input_path}/summaries_train.csv")

#df = pdf.merge(sdf, on="prompt_id")

#train = preprocessor.run(pdf, sdf, mode="train")
train = preprocessor.run(pdf, sdf, mode='test')


# 4 prompt ids, 4 folds
id2fold = {
    "814d6b": 0,
    "39c16e": 1,
    "3b9047": 2,
    "ebad26": 3,
}

train["fold"] = train["prompt_id"].map(id2fold)
#test = preprocessor.run(prompts_test, summaries_test, mode="test")

#test.head()

100%|██████████| 7165/7165 [00:00<00:00, 63584.35it/s]


## Deberta Model

In [9]:
def tokenize_inf(example, tokenizer, config):
    sep = tokenizer.sep_token;
    prompt = sep.join([example["prompt_title"], example["prompt_text"], example["prompt_question"]])
    tokenized = tokenizer(
        example["text"],
        prompt,
        padding=False,
        truncation=True,
        max_length=config.max_seq_length,
    )

    return {
        **tokenized
    }

model_paths = [
    '/root/autodl-tmp/output_fold0_seed42_2609',
    '/root/autodl-tmp/output_fold1_seed42_2609',
    '/root/autodl-tmp/output_fold2_seed42_2609',
    '/root/autodl-tmp/output_fold3_seed42_2609'
]
tokenizer = AutoTokenizer.from_pretrained(model_paths[0]);
model_config = AutoConfig.from_pretrained(model_paths[0]);
model_config.update({
    'hidden_dropout_prob': 0,
    'attention_probs_dropout_prob': 0,
    'num_labels':2,
    'problem_type': 'regression',
    'max_position_embeddings': 1600,
});

data_collator = DataCollatorWithPadding(
    tokenizer = tokenizer,
    pad_to_multiple_of=16,
)

# Do not use pretrained model
deberta_models = []
for model_path in model_paths:
    model = AutoModelForSequenceClassification.from_pretrained(model_path, config = model_config);
    deberta_models.append(model)
config = Config()

### Training

In [12]:
from xgboost import XGBRegressor

In [13]:
from transformers import TrainingArguments 


targets = ["content", "wording"]
drop_columns = ["fold", "student_id", "prompt_id", "text", 
                "prompt_question", "prompt_title", 
                "prompt_text"
               ] + targets

lgb_models = []

for fold in range(2, CFG.n_folds):
    
    X_train_cv = train[train["fold"] != fold]
    X_eval_cv = train[train["fold"] == fold]
    
    ## Create dataset for deberta
    X_train_ds = datasets.Dataset.from_pandas(X_train_cv);
    tokenized_X_train_ds = X_train_ds.map(tokenize_inf, 
                                          batched = False, 
                                          num_proc = 4,
                                          fn_kwargs= {'tokenizer': tokenizer, 'config': config}
                                         );
    X_eval_ds = datasets.Dataset.from_pandas(X_eval_cv);
    tokenized_X_eval_ds = X_eval_ds.map(tokenize_inf,
                                       batched = False,
                                       num_proc = 4,
                                       fn_kwargs = {'tokenizer': tokenizer, 'config': config}
                                       );
    
    
    ## Run deberta model
    if CFG.run_deberta_model:
        deberta = deberta_models[fold]

        infer_args = TrainingArguments(
            output_dir = './',
            do_train = False,
            do_predict = True,
            per_device_eval_batch_size = 16,   
            dataloader_drop_last = False,
            eval_accumulation_steps=1,
        );

        # Init deberta predictor
        trainer = Trainer(
            model = deberta,
            args = infer_args,
            data_collator = data_collator,
            tokenizer = tokenizer
        )
        print(f'Run deberta model for fold {fold}')
        deberta_res_train = trainer.predict(tokenized_X_train_ds)[0]
        deberta_res_eval = trainer.predict(tokenized_X_eval_ds)[0]

        X_train_cv['pred_content_score'] = deberta_res_train[:,0]
        X_train_cv['pred_wording_score'] = deberta_res_train[:,1]
    
        X_eval_cv['pred_content_score'] = deberta_res_eval[:, 0]
        X_eval_cv['pred_wording_score'] = deberta_res_eval[:, 1]
        
        ## Output the files
        X_train_cv.to_csv(f'./boost_input/X_train_cv_{fold}.csv', index = False);
        X_eval_cv.to_csv(f'./boost_input/X_eval_cv_{fold}.csv', index = False);
    else:
        X_train_cv = pd.read_csv(f'./boost_input/X_train_cv_{fold}.csv')
        X_eval_cv = pd.read_csv(f'./boost_input/X_eval_cv_{fold}.csv');
    
    
    ## Train xgboost
    evaluation_results = {}
    eval_labels = {}
    eval_preds = {}
    xgboost_pair = {}
    for target in targets:
        y_train_cv = train[train["fold"] != fold][target]
        y_eval_cv = train[train["fold"] == fold][target]
        
        xgboost_model = XGBRegressor(
            n_estimators=10000, 
            max_depth=64, 
            eta=0.1, 
            subsample=0.9, 
            colsample_bytree=0.8)
        xgboost_model.fit(
            X_train_cv.drop(columns=drop_columns).values,
            y_train_cv.values.reshape((-1, 1)) 
        )
        
        xgboost_pair[target] = xgboost_model
        ## save model
        xgboost_model.save_model(f'./boost_input/xgboost_{target}_{fold}.txt')
        eval_preds[target] = xgboost_model.predict(X_eval_cv.drop(columns=drop_columns).values)
        eval_labels[target] = y_eval_cv
    ## Get compute_mcrmse
    preds_arr = np.concatenate([
        eval_preds['content'].reshape(-1, 1),
        eval_preds['wording'].reshape(-1, 1)],
        axis = 1
    )
    preds_arr_deberta = np.concatenate([
        X_eval_cv[:, 'pred_content_score'].reshape(-1, 1),
        X_eval_cv[:, 'pred_wording_score'].reshape(-1, 1)
    ], axis = 1)
    
    labels_arr = np.concatenate([
        train.loc[train["fold"] == fold, 'content'].values.reshape(-1, 1),
        train.loc[train['fold'] == fold, 'wording'].values.reshape(-1, 1)],
        axis = 1
    )
    
    print(f'deberta compute_mcrmse: ')
    print(compute_mcrmse([preds_arr_deberta, labels_arr]))
    print(f'Print compute_mcrmse for fold {fold}')
    print(compute_mcrmse([preds_arr, labels_arr]))
    print()

Run deberta model for fold 2


AttributeError: 'Series' object has no attribute 'reshape'

In [25]:
X_eval_cv.loc[:, 'pred_content_score'].values.reshape(-1, 1).shape

(2009, 1)

In [29]:
X_train_cv.drop(columns=drop_columns).head(3)

,summary_length,splling_err_num,prompt_length,length_ratio,word_overlap_count,bigram_overlap_count,trigram_overlap_count,quotes_count,pred_content_score,pred_wording_score
0,69,5,671,0.102832,0,5,0,0,0.011516,0.786554
1,56,2,1137,0.049252,0,22,10,0,-0.353386,0.307327
4,253,29,671,0.377049,1,27,5,4,2.434699,2.672835


In [45]:
del xgboost_model, eval_preds

In [46]:
eval_labels = {}
eval_preds = {}
xgboost_pair = {}
for target in targets:
    y_train_cv = train[train["fold"] != fold][target]
    y_eval_cv = train[train["fold"] == fold][target]

    xgboost_model = XGBRegressor(
            n_estimators=1000, 
            max_depth=7, 
            eta=0.001, 
            subsample=0.7, 
            colsample_bytree=0.8)
    xgboost_model.fit(
        X_train_cv.drop(columns=drop_columns).values,
        y_train_cv.values.reshape((-1, 1)) 
    )

    xgboost_pair[target] = xgboost_model
    ## save model
    xgboost_model.save_model(f'./boost_input/xgboost_{target}_{fold}.txt')
    eval_preds[target] = xgboost_model.predict(X_eval_cv.drop(columns=drop_columns).values)
    eval_labels[target] = y_eval_cv
## Get compute_mcrmse
preds_arr = np.concatenate([
    eval_preds['content'].reshape(-1, 1),
    eval_preds['wording'].reshape(-1, 1)],
    axis = 1
)
preds_arr_deberta = np.concatenate([
    X_eval_cv.loc[:, 'pred_content_score'].values.reshape(-1, 1),
    X_eval_cv.loc[:, 'pred_wording_score'].values.reshape(-1, 1)
], axis = 1)

labels_arr = np.concatenate([
    train.loc[train["fold"] == fold, 'content'].values.reshape(-1, 1),
    train.loc[train['fold'] == fold, 'wording'].values.reshape(-1, 1)],
    axis = 1
)

print(f'deberta compute_mcrmse: ')
print(compute_mcrmse([preds_arr_deberta, labels_arr]))
print(f'Print compute_mcrmse for fold {fold}')
print(compute_mcrmse([preds_arr, labels_arr]))
print()

deberta compute_mcrmse: 
{'content_rmse': 0.43378799291648845, 'wording_rmse': 0.5737806059579986, 'mcrmse': 0.5037842994372436}
Print compute_mcrmse for fold 2
{'content_rmse': 0.46537228830234983, 'wording_rmse': 0.6152083574774982, 'mcrmse': 0.5402903228899241}



## Obsolete

In [25]:
eval_labels_dict = {}
for target in targets:
    evaluation_results[target] = lgb_model_pair[target].predict(X_eval_cv.drop(columns=drop_columns))
    eval_labels_dict[target] = train[train["fold"] == fold][target]

In [26]:
eval_preds = np.concatenate([
    evaluation_results['content'].reshape(-1, 1),
    evaluation_results['wording'].reshape(-1, 1)],
    axis = 1)
eval_labels = np.concatenate([
    train.loc[train["fold"] == fold, 'content'].values.reshape(-1, 1),
    train.loc[train['fold'] == fold, 'wording'].values.reshape(-1, 1)],
    axis = 1
)

In [27]:
compute_mcrmse([eval_preds, eval_labels])


{'content_rmse': 0.41287069842384666,
 'wording_rmse': 0.5018779304633396,
 'mcrmse': 0.4573743144435931}

In [11]:
# train_ds = datasets.Dataset.from_pandas(train)
# tokenized_train_ds = train_ds.map(tokenize_inf, batched = False, num_proc = 4, fn_kwargs= {'tokenizer': tokenizer, 'config': config})

In [12]:
from transformers import TrainingArguments
infer_args = TrainingArguments(
    output_dir = './',
    do_train = False,
    do_predict = True,
    per_device_eval_batch_size = 16,   
    dataloader_drop_last = False,
    eval_accumulation_steps=1,
)

# init trainer
trainer = Trainer(
    model = model, 
    args = infer_args,
    data_collator = data_collator,
    tokenizer = tokenizer
)

deberta_res = []
for model in models:
    deberta_res.append(trainer.predict(tokenized_train_ds)[0])

In [13]:
deberta_res_mean = np.mean(np.array(deberta_res), axis = 0)

In [14]:
train['pred_content_score'] = deberta_res_mean[:,0]
train['pred_wording_score'] = deberta_res_mean[:,1]
#train.to_csv('./train_2.csv', index = False)

In [15]:
#train = pd.read_csv('./train_2.csv')

In [16]:
def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }


mcrmse = compute_mcrmse((train.loc[:, ['pred_content_score', 'pred_wording_score']].values, 
               train.loc[:, ['content', 'wording']].values))
mcrmse

{'content_rmse': 0.3729688675080504,
 'wording_rmse': 0.4903757307947375,
 'mcrmse': 0.43167229915139393}

In [19]:
train.loc[:, ['pred_content_score', 'pred_wording_score', 'content', 'wording']].sample(5)

,pred_content_score,pred_wording_score,content,wording
2839,-0.305602,0.348631,-0.093814,0.503833
1768,-0.425709,-0.498831,-0.627647,-0.125597
4296,-0.335351,-0.187537,-0.296443,-1.133453
5288,1.678848,1.327419,0.686362,0.704364
5314,-0.706576,-0.307890,-0.776512,-0.383759


### Lightgbm Model

In [21]:
targets = ["content", "wording"]

drop_columns = ["fold", "student_id", "prompt_id", "text", 
                "prompt_question", "prompt_title", 
                "prompt_text"
               ] + targets

In [22]:
lgb_model_dict = {}

for target in targets:
    lgb_models = []
    
    #for fold in range(CFG.n_folds):
    for fold in range(3,4):

        X_train_cv = train[train["fold"] != fold].drop(columns=drop_columns)
        y_train_cv = train[train["fold"] != fold][target]

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
        dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)

        params = {
                  'boosting_type': 'gbdt',
                  'random_state': 42,
                  'objective': 'regression',
                  'metric': 'rmse',
                  'learning_rate': 0.05,
                  }

        evaluation_results = {}
        lgb_model = lgb.train(params,
                          num_boost_round=10000,
                            #categorical_feature = categorical_features,
                          valid_names=['train', 'valid'],
                          train_set=dtrain,
                          valid_sets=dval,
                          callbacks=[
                              lgb.early_stopping(stopping_rounds=30, verbose=True),
                               lgb.log_evaluation(100),
                              lgb.callback.record_evaluation(evaluation_results)
                            ],
                          )
        lgb_model.save_model(f'./lgbt_{target}_{fold}.txt')
        lgb_models.append(lgb_model)
    
    lgb_model_dict[target] = lgb_models

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000719 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1347
[LightGBM] [Info] Number of data points in the train set: 5169, number of used features: 10
[LightGBM] [Info] Start training from score 0.013356
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[42]	train's rmse: 0.413015
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000403 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1347
[LightGBM] [Info] Number of data points in the train set: 5169, number of used features: 10
[LightGBM] [Info] Start training from score 0.028040
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[38]	train's rmse: 0.503381


In [14]:
for target in targets:
    lgb_model_dict[target].save_model(f'./lgbt_{target}_{idx}.txt')

NameError: name 'lgb_model_dict' is not defined

## CV score

In [29]:
### test load
model_targets = ['content', 'wording']

lgb_model_dict = {}
for model_target in model_targets:
    lgb_model_dict[model_target] = [];
    for idx in range(4):
        lgb_model_dict[model_target].append(lgb.Booster(model_file = f'./lgbt_{model_target}_{idx}.txt'))

In [25]:
preds_content = lgb_model_dict['content'][0].predict(train[train['fold'] == 3].drop(columns = drop_columns))
wording_content = lgb_model_dict['wording'][0].predict(train[train['fold'] == 3].drop(columns = drop_columns))

In [27]:
compute_mcrmse(
    (np.concatenate([preds_content.reshape(-1, 1), wording_content.reshape(-1, 1)], axis = 1),
    train.loc[train['fold'] == 3, ['content', 'wording']])
)

{'content_rmse': 0.4130148003572234,
 'wording_rmse': 0.5033809768388605,
 'mcrmse': 0.45819788859804195}

In [20]:
from sklearn.metrics import mean_squared_error
# cv
rmses = []

for target in targets:
    lgb_models = lgb_model_dict[target]

    preds = []
    trues = []
    
    for fold, lgb_model in enumerate(lgb_models):
        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        pred = lgb_model.predict(X_eval_cv)

        trues.extend(y_eval_cv)
        preds.extend(pred)
        
    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")

content_rmse : 0.37573650633040434
wording_rmse : 0.4850980756412646
mcrmse : 0.4304172909858345


### Check mcrmse on training data

In [21]:
lgb_model_content_0 = lgb_model_dict['content'][0]
lgb_model_wording_0 = lgb_model_dict['wording'][0]

In [31]:
train_content_res = lgb_model_content_0.predict(train.drop(columns = drop_columns)).reshape(-1,1)
train_wording_res = lgb_model_wording_0.predict(train.drop(columns = drop_columns)).reshape(-1,1)
preds = np.concatenate([train_content_res, train_wording_res], axis = 1)
preds

array([[ 0.09165974,  0.89087325],
       [-0.7616748 , -0.50564852],
       [ 2.68807138,  2.58113256],
       ...,
       [-1.07972116, -0.60669181],
       [-0.09735215,  0.38939329],
       [ 1.06160919,  0.55252552]])

In [35]:
labels = train.loc[:, ['content', 'wording']]
compute_mcrmse([preds, labels])

{'content_rmse': 0.32244564104233536,
 'wording_rmse': 0.42940207314241746,
 'mcrmse': 0.3759238570923764}

### Save the model

In [31]:
import joblib
for target in targets:
    idx = 0;
    for lgb_model in lgb_model_dict[target]:
        lgb_model.save_model(f'./lgbt_{target}_{idx}.txt')
        idx+=1

In [32]:
lgb_model_dict[target]